In [1]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, sqlite3, pickle, time, datetime, random

from sklearn.model_selection import LeaveOneOut
import os

t = time.time()

# 0. Load Data a Parameters

In [2]:
#######################################
### Relevant fields for pre-processing
#######################################
reference_date = datetime.datetime(1970,1,1)

##### Fields in table patients ##########
patient_field         = 'patient_id'
age_field             = 'age_at_admission\r'
sex_field             = 'sex'
ethnic_field          = 'ethnic_origin'
death_ind_field       = 'death_indicator'
death_date_field      = 'date_of_death'
death_date_code_field = 'date_code_of_death'
mort_in_hosp_field    = 'Thirty_day_mort'

##### Fields in table admissions ########
admission_field           = 'admission_id'
diagnosis_field           = 'episode_diagnoses'
admn_date_field           = 'admission_date_time'
admn_discharge_field      = 'discharge_date_time'
admn_date_code_field      = 'admission_date_code_time'
admn_discharge_code_field = 'discharge_date_code_time'
lengthofstay_field        = 'lengthofstay'
isPneumonia_field         = 'isPneumonia'
mortal_admin_field        = 'mortal_admin'
comorbidity_field         = 'Comorbidity_score'
icu_admin_field           = 'icu_count\r'
no_eobs_field             = 'no_obs_eobs'
no_haematology_field      = 'no_haematology_eobs'

 ##### Fields in table eObservations #####
eObs_time_field      = 'timestamp'
eObs_time_code_field = 'timestamp_code'
eObs_time_prev_obs   = 'time_since_prev_obs_in_mins'
eObs_resprate_field  = 'rr'
eObs_sbp_field       = 'sbp'
eObs_dbp_field       = 'dbp'
eObs_newscore_field  = 'ews'
eObs_heartrate_field = 'heart_rate'
eObs_temptr_field    = 'temperature\r'
eObs_oxygen_field    = 'Oxygen_Saturation'

 ##### Fields in table haematological tests #####
test_code_field      = 'local_test_code'
test_time_field      = 'sample_collection_date_time'
test_time_code_field = 'sample_collection_date_code_time'

### 0.1. Load Data

In [3]:
##############################################################################
# 0. LOAD DATA AND PARAMETERS
##############################################################################
data = pickle.load( open('DataFrame_pickles/df_patients_admissions_v3.pickle', 'rb'))
df_patients   = data[0]
df_admissions = data[1]
data = pickle.load( open('DataFrame_pickles/df_haematology_v1.pickle', 'rb'))
df_haematology  = data[0]

list_admissions      = df_admissions[admission_field].unique().tolist()
list_haemat_features = list( df_haematology.columns[2:])

In [4]:
df_haematology.dtypes

admission_id                            int64
sample_collection_date_time    datetime64[ns]
CREA                                  float64
UREA                                  float64
K                                     float64
EGFR                                  float64
GFR                                   float64
WBC                                   float64
PLT                                   float64
HCT                                   float64
HGB                                   float64
RBC                                   float64
MCH                                   float64
MCV                                   float64
NEUAB                                 float64
TLYMAB                                float64
EOSAB                                 float64
MONAB                                 float64
BASAB                                 float64
ALB                                   float64
ALP                                   float64
BILI                              

In [5]:
df_haematology[df_haematology[admission_field]==99155969].head(4)

,admission_id,sample_collection_date_time,CREA,UREA,K,EGFR,GFR,WBC,PLT,HCT,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
67298,99155969,2018-02-13 08:09:00,79.0,4.3,3.8,NaN,65.0,14.6,270.0,0.379,...,25.9,81.0,13.08,0.48,0.06,0.89,0.07,46.0,95.0,10.0
67299,99155969,2018-02-14 18:16:00,80.0,4.0,4.6,NaN,64.0,10.2,246.0,0.352,...,26.1,82.0,8.98,0.46,0.14,0.64,0.02,NaN,NaN,NaN
67300,99155969,2018-02-16 10:03:00,68.0,3.7,4.0,NaN,78.0,12.0,267.0,0.346,...,25.9,82.0,10.76,0.70,0.07,0.49,0.02,NaN,NaN,NaN
67301,99155969,2018-02-17 08:08:00,66.0,2.8,3.5,NaN,80.0,12.5,252.0,0.348,...,25.7,81.0,10.33,1.23,0.22,0.65,0.03,NaN,NaN,NaN


# 1. Definition of Function for interpolation

In [6]:
##############################################################################
# 1. DEFINITION OF FUNCTIONS
##############################################################################
# 1.1. interpolate_by_method ===========================================
def interpolate_by_method(series, method, plot = 0):
    if ('polynomial' in method) or ('nearest' in method):
        interpolate = series.interpolate(method = method[:-1], order = int(method[-1]))
    else:
        interpolate = series.interpolate(method = method, order = method)
    if plot != 0: interpolate.plot()
    return interpolate
# 1.2. cross_validation_series ===========================================
def cross_validation_series(feature_series, times, interp_methods, upsamptimes):    
    cv     = LeaveOneOut()
    errors = np.zeros([len(feature_series),len(interp_methods)])
    h = 0
    for train_ix, test_ix in cv.split(times):
        
        feat_train, time_train = feature_series.iloc[train_ix], times.iloc[train_ix]
        feat_test, time_test   = feature_series.iloc[test_ix], times.iloc[test_ix]
             
        values      = [feat_train[time_train == tm].mean() for i,tm in enumerate(upsamptimes)]
        cv_series   = pd.Series(values, index=upsamptimes)
        
        for idx_method, method in enumerate(interp_methods):
            if (cv_series.count() < 4  and method == 'polynomial3') or (method == 'polynomial2' and cv_series.count() < 3): 
                errors[h, idx_method] = 1
                continue
            interp = interpolate_by_method(cv_series, method)
            errors[h, idx_method] = (interp.loc[time_test].values[0] - feat_test.values[0]) ** 2
        h = h + 1    
        
    errors  = np.nanmean(errors, axis=0)
    errors  = np.sqrt(errors) # RMSE
    error   = errors.min()
    methods = [interp_methods[i] for i, x in enumerate(errors == np.nanmin(errors)) if x]
    return methods[0], errors

# 2. Construction of df_old 
### Counts the number of samples of each admission and count the bt available in each admission. 
### to get the number of admissions to interpolate

In [7]:
a = [admission_field] + list_haemat_features + ['totals']
print(len(a))
print(len(list_haemat_features))

list_haemat_features

22
20


['CREA',
 'UREA',
 'K',
 'EGFR',
 'GFR',
 'WBC',
 'PLT',
 'HCT',
 'HGB',
 'RBC',
 'MCH',
 'MCV',
 'NEUAB',
 'TLYMAB',
 'EOSAB',
 'MONAB',
 'BASAB',
 'ALB',
 'ALP',
 'BILI']

In [8]:
t2 = time.time()
list_admissions_haemt  = df_haematology[admission_field].unique().tolist()
df_old = []
for admin in list_admissions_haemt:
    row = [admin]
    temp = df_haematology[(df_haematology[admission_field] == admin)].copy()
    for test in list_haemat_features:
        row.append(temp[test].notnull().sum())
    row.append(len(temp))
    df_old.append(row)
df_old = pd.DataFrame(df_old, columns = [admission_field] + list_haemat_features + ['totals'])
display(df_old.head(5))
print('Elapsed in extracting haematology information:', time.time()-t2)

,admission_id,CREA,UREA,K,EGFR,GFR,WBC,PLT,HCT,HGB,...,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI,totals
0,30861976,14,14,14,0,7,14,14,14,14,...,14,13,13,13,13,13,4,4,4,14
1,75609064,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,5,5,5,7
2,53466902,18,18,18,0,18,19,19,19,19,...,19,19,19,19,19,19,14,14,12,23
3,48308718,10,10,10,0,10,8,8,8,8,...,8,8,8,8,8,8,4,4,3,11
4,31202941,4,4,4,0,4,4,4,4,4,...,4,4,4,4,4,4,1,1,1,4


Elapsed in extracting haematology information: 63.35138511657715


In [9]:
##############################################################################
# LIST OF ADMISSIONS TO INTERPOLATE
# Depending on length of stay and have more than threshold cut observations
##############################################################################
los_cut_threshold = 1 # Threshold in days of length of stay
obs_cut_threshold = 2 # Threshold of number of observations per admissions
admissions_los_thr = df_admissions[df_admissions[lengthofstay_field] >= datetime.timedelta(days=los_cut_threshold)][admission_field]
admissions_m1t_obs = df_old[df_old['totals'] >  obs_cut_threshold][admission_field]
admissions_interp  = list(set(admissions_los_thr) & set(admissions_m1t_obs))

print("number of admissions to interpolate", len(admissions_interp))

number of admissions to interpolate 12066


# 3. Interpolation of Haematology

In [10]:
t = time.time()
new_haematology_3d = pd.DataFrame(columns = df_haematology.columns)
new_haematology_2d = pd.DataFrame(columns = df_haematology.columns)
new_haematology_1d = pd.DataFrame(columns = df_haematology.columns)
admins_pass_1 = []
admins_pass_2 = []
for idx, adm in enumerate(admissions_interp):
    
    t1 = time.time()
    
    adm_los  = df_admissions[df_admissions[admission_field] == adm].iloc[0][lengthofstay_field]
    adm_haem = df_haematology[df_haematology[admission_field]==adm].copy()    

    times          = adm_haem[test_time_field]
    
    times_3d = times[times<= (times.min() + datetime.timedelta(days=3))]
    times_3d = times.iloc[:len(times_3d) + 1]
    
    if (adm_los.days < 1):
        admins_pass_1.append(adm)
    elif (len(times_3d) < 3) : 
        admins_pass_2.append(adm)
        continue
    #round to the closest quarter of hour
    times_3d = times_3d.apply(lambda dt: datetime.datetime(dt.year, dt.month, 
                                                       dt.day, 
                                                       min(round(dt.hour/6)*6,18),
                                                       0))
    upsamp_times    = pd.date_range(times_3d.min(), times_3d.max(), freq = '6H').tolist()
    upsamp_features = np.zeros([len(upsamp_times),len(list_haemat_features)])
    
    for idx_feat, feature in enumerate(list_haemat_features):
        #if feature == 'ALB':break
        feature_series = adm_haem[feature].iloc[:len(times_3d)]    
        values  = [feature_series[times_3d == tm].mean() for i,tm in enumerate(upsamp_times)]        
        series_ = pd.Series(values, index=upsamp_times)
        
        if series_.notnull().sum() == 0: # If all of them are missing skip feature
            upsamp_features[:, idx_feat] = np.nan
            continue            
        elif series_.notnull().sum() == 1: # If there is only one value leave all the same value
            upsamp_features[:, idx_feat] = series_[series_.notnull()].iloc[0]
            continue
        
        interp  = interpolate_by_method(series_, 'linear') #'pchip')
        if interp.isnull().sum() > 0: interp = interp.interpolate(method='backfill', limit =20, limit_direction = 'backward')            
        upsamp_features[:, idx_feat] = interp
        
    df = pd.DataFrame(upsamp_features, columns = list_haemat_features)
    df[admission_field] = adm   
    df[test_time_field] = upsamp_times
    
    if (adm_los.days >= 3) and (len(df) >= 4*3): new_haematology_3d  = pd.concat([new_haematology_3d, df.iloc[:4*3]])
    if (adm_los.days >= 2) and (len(df) >= 4*2): new_haematology_2d  = pd.concat([new_haematology_2d, df.iloc[:4*2]])
    if (adm_los.days >= 1) and (len(df) >= 4*1): new_haematology_1d  = pd.concat([new_haematology_1d, df.iloc[:4*1]])
    print(idx, 'Admission: ',adm, ' is ', time.time() - t1)
print('Elapsed in extracting haematology information:', time.time()-t)

0 Admission:  99155969  is  0.1233820915222168
2 Admission:  87883782  is  0.1408064365386963
3 Admission:  89653261  is  0.12340402603149414
6 Admission:  61014043  is  0.11631488800048828
7 Admission:  27426845  is  0.13361883163452148
9 Admission:  88080417  is  0.1623387336730957
10 Admission:  99549221  is  0.14073443412780762
11 Admission:  77791269  is  0.1564922332763672
12 Admission:  24870960  is  0.23333334922790527
13 Admission:  43909169  is  0.13215398788452148
14 Admission:  35979313  is  0.15697169303894043
15 Admission:  49479735  is  0.16456818580627441
16 Admission:  86868024  is  0.13618183135986328
17 Admission:  51511355  is  0.12143874168395996
18 Admission:  56885308  is  0.11504673957824707
19 Admission:  91390013  is  0.16557860374450684
20 Admission:  82640967  is  0.12915825843811035
22 Admission:  42795082  is  0.1351325511932373
23 Admission:  36634708  is  0.13121724128723145
24 Admission:  12451925  is  0.13787412643432617
25 Admission:  29687897  is  0.

188 Admission:  96666187  is  0.12041831016540527
189 Admission:  39715405  is  0.11384320259094238
190 Admission:  32670287  is  0.13504934310913086
191 Admission:  34701905  is  0.19705605506896973
192 Admission:  64553556  is  0.17085909843444824
193 Admission:  45646421  is  0.13560080528259277
194 Admission:  79364693  is  0.12657451629638672
195 Admission:  47284837  is  0.10553216934204102
196 Admission:  64488038  is  0.2346818447113037
197 Admission:  8520296  is  0.12910223007202148
198 Admission:  53445229  is  0.13538670539855957
199 Admission:  16384623  is  0.16184401512145996
200 Admission:  27198063  is  0.1263129711151123
202 Admission:  38273653  is  0.11579084396362305
203 Admission:  95715958  is  0.09012365341186523
204 Admission:  27722364  is  0.15323185920715332
205 Admission:  12059266  is  0.11692667007446289
206 Admission:  27329155  is  0.10748815536499023
207 Admission:  48759429  is  0.10070180892944336
208 Admission:  80183942  is  0.16123390197753906
210

378 Admission:  46859386  is  0.369966983795166
379 Admission:  23397498  is  0.14156770706176758
381 Admission:  54035579  is  0.3745391368865967
382 Admission:  45843589  is  0.354933500289917
383 Admission:  51217541  is  0.3029453754425049
384 Admission:  84968581  is  0.20471620559692383
385 Admission:  59704460  is  0.2314155101776123
386 Admission:  12289167  is  0.18095183372497559
387 Admission:  34079888  is  0.4710693359375
388 Admission:  87721105  is  0.13513827323913574
389 Admission:  66978961  is  0.12194061279296875
391 Admission:  73925780  is  0.12898850440979004
392 Admission:  29263002  is  0.1337122917175293
393 Admission:  39781538  is  0.1479182243347168
394 Admission:  11011235  is  0.16467881202697754
395 Admission:  16876709  is  0.14264392852783203
396 Admission:  86115497  is  0.19566082954406738
399 Admission:  42796207  is  0.11957955360412598
400 Admission:  78742704  is  0.1518843173980713
401 Admission:  15140017  is  0.16107439994812012
403 Admission:

570 Admission:  23529038  is  0.189894437789917
571 Admission:  71698005  is  0.14070630073547363
572 Admission:  96372314  is  0.1406540870666504
573 Admission:  36013659  is  0.15635180473327637
574 Admission:  3868253  is  0.17257404327392578
575 Admission:  33916510  is  0.18566322326660156
576 Admission:  9668190  is  0.11962532997131348
578 Admission:  89589344  is  0.13877511024475098
579 Admission:  63145571  is  0.2161257266998291
580 Admission:  39421542  is  0.14801025390625
581 Admission:  24807015  is  0.09399962425231934
582 Admission:  64915052  is  0.23007583618164062
584 Admission:  69109364  is  0.13083839416503906
586 Admission:  88114807  is  0.13774394989013672
587 Admission:  31327866  is  0.11826491355895996
588 Admission:  10421883  is  0.12142586708068848
590 Admission:  70125183  is  0.13291645050048828
591 Admission:  43517568  is  0.13361763954162598
592 Admission:  44271238  is  0.15878844261169434
593 Admission:  35456647  is  0.18547558784484863
594 Admis

770 Admission:  21006415  is  0.12429046630859375
771 Admission:  75008082  is  0.1559429168701172
772 Admission:  78645333  is  0.13335680961608887
773 Admission:  96241750  is  0.1884775161743164
774 Admission:  44632149  is  0.09621787071228027
777 Admission:  96897116  is  0.15498924255371094
778 Admission:  57444446  is  0.13574457168579102
779 Admission:  41388127  is  0.22843599319458008
780 Admission:  86116446  is  0.18765735626220703
781 Admission:  42764387  is  0.14189910888671875
782 Admission:  24414308  is  0.12350201606750488
783 Admission:  31623275  is  0.06596755981445312
784 Admission:  69175405  is  0.13395380973815918
785 Admission:  76843117  is  0.15472006797790527
786 Admission:  18647151  is  0.1717684268951416
787 Admission:  70092912  is  0.1315009593963623
788 Admission:  69896305  is  0.23110437393188477
789 Admission:  96340088  is  0.16620755195617676
790 Admission:  58525818  is  0.12248492240905762
791 Admission:  72747138  is  0.18371224403381348
793 

...

11222 Admission:  77199121  is  0.26320457458496094
11224 Admission:  83556115  is  0.2417740821838379
11225 Admission:  62453533  is  0.39794468879699707
11227 Admission:  36042530  is  0.24386024475097656
11228 Admission:  56129315  is  0.2578086853027344
11229 Admission:  94697252  is  0.2419114112854004
11230 Admission:  34174758  is  0.27150464057922363
11231 Admission:  54785832  is  0.32779765129089355
11232 Admission:  12351273  is  0.2604374885559082
11235 Admission:  58095411  is  0.2639338970184326
11236 Admission:  1439548  is  0.2372751235961914
11237 Admission:  98367295  is  0.26945948600769043
11239 Admission:  32175940  is  0.28305530548095703
11240 Admission:  58783558  is  0.2542917728424072
11241 Admission:  61339462  is  0.22454619407653809
11242 Admission:  44857162  is  0.2696349620819092
11243 Admission:  31258454  is  0.2375471591949463
11244 Admission:  85096279  is  0.2827463150024414
11245 Admission:  80639832  is  0.2533278465270996
11246 Admission:  128428

11401 Admission:  96631009  is  0.2985403537750244
11402 Admission:  66156770  is  0.27068161964416504
11403 Admission:  72120550  is  0.23931002616882324
11404 Admission:  59111654  is  0.2547159194946289
11405 Admission:  12581100  is  0.22490859031677246
11406 Admission:  87881967  is  0.2841775417327881
11407 Admission:  81688820  is  0.2472679615020752
11408 Admission:  28244214  is  0.2541484832763672
11409 Admission:  97122551  is  0.26662778854370117
11410 Admission:  62093559  is  0.31423020362854004
11411 Admission:  74512633  is  0.3087775707244873
11412 Admission:  86898935  is  0.08089947700500488
11413 Admission:  39254268  is  0.2892739772796631
11414 Admission:  41384188  is  0.24437379837036133
11415 Admission:  21199102  is  0.2601771354675293
11416 Admission:  60815638  is  0.2797544002532959
11417 Admission:  44300570  is  0.2859067916870117
11418 Admission:  71366938  is  0.3459765911102295
11419 Admission:  22673692  is  0.3414492607116699
11420 Admission:  555399

11581 Admission:  57604838  is  0.2562124729156494
11582 Admission:  13564647  is  0.26196932792663574
11584 Admission:  82279150  is  0.23670434951782227
11585 Admission:  18938607  is  0.25948381423950195
11586 Admission:  80542447  is  0.27503037452697754
11587 Admission:  1866481  is  0.27593135833740234
11588 Admission:  86473457  is  0.28420376777648926
11589 Admission:  93125362  is  0.2669847011566162
11590 Admission:  22903537  is  0.2669408321380615
11591 Admission:  8387313  is  0.2838423252105713
11592 Admission:  31554294  is  0.31146907806396484
11593 Admission:  71564015  is  0.2806544303894043
11594 Admission:  22543097  is  0.24534893035888672
11595 Admission:  23755519  is  0.25563764572143555
11596 Admission:  4258564  is  0.2817564010620117
11597 Admission:  71006987  is  0.25628161430358887
11598 Admission:  94665487  is  0.30255937576293945
11601 Admission:  18217760  is  0.2677919864654541
11603 Admission:  47348520  is  0.2882380485534668
11604 Admission:  33356

11764 Admission:  54885627  is  0.3569209575653076
11765 Admission:  99384578  is  0.283266544342041
11766 Admission:  78839043  is  0.3056163787841797
11767 Admission:  21429510  is  0.27451157569885254
11768 Admission:  50494726  is  0.3066980838775635
11769 Admission:  82443527  is  0.3130221366882324
11770 Admission:  94928142  is  0.2635526657104492
11772 Admission:  80903446  is  0.275484561920166
11773 Admission:  52067608  is  0.27845048904418945
11774 Admission:  19922203  is  0.46498990058898926
11775 Admission:  66354460  is  0.4040212631225586
11776 Admission:  41188639  is  0.42412662506103516
11777 Admission:  60751138  is  0.26145267486572266
11778 Admission:  10714406  is  0.1561577320098877
11779 Admission:  37846321  is  0.27761387825012207
11780 Admission:  87620919  is  0.25266599655151367
11781 Admission:  9141560  is  0.5575969219207764
11782 Admission:  31522115  is  0.2806515693664551
11783 Admission:  67140931  is  0.5024549961090088
11784 Admission:  6159684  

11941 Admission:  53346018  is  0.2668752670288086
11942 Admission:  46005998  is  0.2605583667755127
11943 Admission:  78544624  is  0.27974820137023926
11945 Admission:  27918066  is  0.2871384620666504
11946 Admission:  7274229  is  0.2894110679626465
11947 Admission:  91356918  is  0.26572537422180176
11949 Admission:  46464763  is  0.3142552375793457
11950 Admission:  16023297  is  0.2603733539581299
11951 Admission:  17104645  is  0.25650715827941895
11952 Admission:  13303559  is  0.3028864860534668
11954 Admission:  58556171  is  0.2647075653076172
11955 Admission:  25132813  is  0.3041069507598877
11957 Admission:  36536078  is  0.29842329025268555
11958 Admission:  22413072  is  0.2751784324645996
11959 Admission:  12877590  is  0.2988457679748535
11960 Admission:  53051160  is  0.2828528881072998
11962 Admission:  52363033  is  0.27431392669677734
11963 Admission:  81788699  is  0.25572967529296875
11964 Admission:  66879261  is  0.2893798351287842
11965 Admission:  32571168

In [11]:
print("Number of admissions not processed that were not well filtered by LoS")
print(len(admins_pass_1))
print("Number of admissions with less than 2 observations to be interpolated:")
print(len(admins_pass_2))

Number of admissions not processed that were not well filtered by LoS
0
Number of admissions with less than 2 observations to be interpolated:
1540


In [12]:
print("len 1d:", len(new_haematology_1d['admission_id'].unique()))
print("len 2d:", len(new_haematology_2d['admission_id'].unique()))
print("len 3d:", len(new_haematology_3d['admission_id'].unique()))

len 1d: 10519
len 2d: 10475
len 3d: 10264


In [13]:
new_haematology_3d.head(4)

,admission_id,sample_collection_date_time,CREA,UREA,K,EGFR,GFR,WBC,PLT,HCT,...,MCH,MCV,NEUAB,TLYMAB,EOSAB,MONAB,BASAB,ALB,ALP,BILI
0,99155969,2018-02-13 06:00:00,79.0,4.3,3.8,NaN,65.0,14.6,270.0,0.379,...,25.9,81.0,13.08,0.48,0.06,0.89,0.07,46.0,95.0,10.0
1,99155969,2018-02-13 12:00:00,79.166667,4.25,3.933333,NaN,64.833333,13.866667,266.0,0.3745,...,25.933333,81.166667,12.396667,0.476667,0.073333,0.848333,0.061667,46.0,95.0,10.0
2,99155969,2018-02-13 18:00:00,79.333333,4.2,4.066667,NaN,64.666667,13.133333,262.0,0.37,...,25.966667,81.333333,11.713333,0.473333,0.086667,0.806667,0.053333,46.0,95.0,10.0
3,99155969,2018-02-14 00:00:00,79.5,4.15,4.2,NaN,64.5,12.4,258.0,0.3655,...,26.0,81.5,11.03,0.47,0.1,0.765,0.045,46.0,95.0,10.0


In [14]:
df = pd.concat([new_haematology_3d.isna().sum(), new_haematology_3d.dtypes], axis=1)
df = df.rename(columns ={0:'nulls',1:'type'})
display(df.sort_values(by=['nulls']))

,nulls,type
admission_id,0,object
MCV,0,object
MCH,0,object
RBC,0,object
HCT,0,object
WBC,0,object
HGB,0,object
K,0,object
UREA,0,object
CREA,0,object


In [15]:
pickle.dump(new_haematology_3d, open( "DataFrame_pickles/new_haematology_3d.pickle", "wb" ))